In [19]:
#Importing libraries
import praw
import pandas as pd
from textblob import TextBlob
from datetime import datetime

#Reddit credentials
reddit = praw.Reddit(
    client_id="DmOlaVsQER7B6QrHpFvj7A",         
    client_secret="A_F8imgZLu8CoX8twzxBMA4K72WEeA",  
    user_agent="SentimentScraper by /u/Balleriner",
    username="Balleriner",
    password="*******"  #redacted for security
)

In [23]:
#Scraper function
def scrape_subreddit(subreddit_name, label, limit=100):
    posts = []
    for submission in reddit.subreddit(subreddit_name).hot(limit=limit):
        text = submission.title + " " + submission.selftext
        posts.append([submission.created_utc, text, label])
    return posts

#Scraping posts
chatgpt_posts = scrape_subreddit("ChatGPT", "ChatGPT")
bard_posts = scrape_subreddit("Bard", "Bard")

#Combining data
data = chatgpt_posts + bard_posts
df = pd.DataFrame(data, columns=["Timestamp", "Post", "Topic"])

In [25]:
#Checking number of posts scraped
print(f"ChatGPT posts: {len(chatgpt_posts)}")
print(f"Bard posts: {len(bard_posts)}")

ChatGPT posts: 100
Bard posts: 100


In [27]:
df.head()

,Timestamp,Post,Topic
0,1.739061e+09,Calling AI Researchers & Startup Founders To J...,ChatGPT
1,1.753149e+09,Why does chatgpt keep doing this? I've tried s...,ChatGPT
2,1.753211e+09,RIP Ozzy Prince of Darkness going into the Lig...,ChatGPT
3,1.753203e+09,just added ChatGPT to my calculator Demo on Yo...,ChatGPT
4,1.753171e+09,Back to the Future as a community theater prod...,ChatGPT


In [29]:
#Polarity + Sentiment classification
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

def classify_sentiment(score):
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

df["Polarity"] = df["Post"].apply(get_polarity)
df["Sentiment"] = df["Polarity"].apply(classify_sentiment)

#Converting timestamp to readable date
df["Date"] = pd.to_datetime(df["Timestamp"], unit="s")

In [31]:
from datetime import datetime

filename = f"reddit_chatgpt_vs_bard_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
df.to_csv(filename, index=False)

print(f"CSV saved as {filename}")

CSV saved as reddit_chatgpt_vs_bard_20250722_1542.csv
